# 아파트 실거래가 분석

In [2]:
import json
import os
import pandas as pd
import requests
import xml.etree.ElementTree as ET

## 1. 실거래 데이터 가져오기

공공데이터 포탈로부터 API를 사용하여 아파트 실거래 데이터를 가져온다.

In [4]:
# API URL
api_url = 'http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev'

# API Key (시스템 환경 변수에 저장된 Key를 가져옴)
api_key = os.environ['DATA_PORTAL_API_KEY']

In [5]:
def parse_xml_items(items):
    """
    응답 XML 데이터로부터 필요 컬럼의 값을 추출
    """
    item_list = []
    
    for item in items:
        item_dict = {}
        
        for item_elem in item:
            if item_elem.tag in ['dealAmount', 'buildYear', 'dealYear', 'dealMonth', 'roadNm', 'umdNm', 'aptDong', 'aptNm', 'excluUseAr', 'floor']:
                item_dict[item_elem.tag] = item_elem.text.strip()

        item_list.append(item_dict)

    return item_list

In [6]:
def parse_xml(data):
    """
    응답 XML 데이터 파싱
    """
    data_dict = {
        'num_of_rows': 0,
        'page_no': 0,
        'total_count': 0,
        'item_list': []
    }
    
    # XML 객체 변환 
    xml_data = ET.fromstring(data)
    
    # 'body' 요소 찾기
    for elem in xml_data:
        if elem.tag == 'body':
            xml_body = elem
            break

    if xml_body:
        for elem in xml_body:
            try:
                if elem.tag == 'numOfRows':
                    # row 개수
                    data_dict['num_of_rows'] = int(elem.text.strip())
                
                elif elem.tag == 'pageNo':
                    data_dict['page_no'] = int(elem.text.strip())

                elif elem.tag == 'totalCount':
                    data_dict['total_count'] = int(elem.text.strip())

                elif elem.tag == 'items':
                    # 필요 컬럼의 값 가져오기
                    data_dict['item_list'] = parse_xml_items(elem)
                
                else:
                    raise Exception('Body Element Tag Error!')
            
            except Exception as e:
                print(e)

    return data_dict

In [7]:
def get_data(year:int, month:int, lawd_cd:str):
    """
    공공데이터포탈에서 데이터 가져오기 
    """
    data_list = []
    page_no = 1

    while range(10):
        # 파라미터 설정
        params ={
            'serviceKey' : api_key,
            'pageNo' : str(page_no),
            'numOfRows' : '100',
            'LAWD_CD' : lawd_cd,
            'DEAL_YMD' : f'{year}{month:02}'
        }
    
        # 데이터 가져오기
        response = requests.get(api_url, params=params)
    
        # 응답 데이터 분석
        data = parse_xml(response.content)

        # 데이터 목록 추가
        data_list += data['item_list']

        if len(data_list) >= data['total_count']:
            print(f"{year}-{month}: page_no: {page_no}, total_count: {data['total_count']}")
            break

        print(f"{year}-{month}: page_no: {page_no}, total_count: {data['total_count']}")

        # 다음 페이지 번호 설정
        page_no += 1

    return data_list

In [8]:
# 2024년도 의왕시 삼동의 실거래가 가져오기
data_list = []
year = 2024
last_month = 11
lawd_cd = '41430'  # 행정동 코드 (https://www.code.go.kr/stdcode/regCodeL.do)

for month in range(1, last_month + 1):
    data_list += get_data(year, month, lawd_cd)

print(len(data_list))

2024-1: page_no: 1, total_count: 100
2024-2: page_no: 1, total_count: 65
2024-3: page_no: 1, total_count: 121
2024-3: page_no: 2, total_count: 121
2024-4: page_no: 1, total_count: 105
2024-4: page_no: 2, total_count: 105
2024-5: page_no: 1, total_count: 123
2024-5: page_no: 2, total_count: 123
2024-6: page_no: 1, total_count: 148
2024-6: page_no: 2, total_count: 148
2024-7: page_no: 1, total_count: 253
2024-7: page_no: 2, total_count: 253
2024-7: page_no: 3, total_count: 253
2024-8: page_no: 1, total_count: 190
2024-8: page_no: 2, total_count: 190
2024-9: page_no: 1, total_count: 95
2024-10: page_no: 1, total_count: 88
2024-11: page_no: 1, total_count: 11
1299


In [12]:
# 응답 데이터를 DataFrame으로 변환
df_apt_price = pd.DataFrame(data_list)

# '거래금액' 데이터를 숫자 타입으로 변환
df_apt_price['dealAmount'] = df_apt_price['dealAmount'].str.replace(',', '').astype(int)

In [13]:
df_apt_price.head()

,aptDong,aptNm,buildYear,dealAmount,dealMonth,dealYear,excluUseAr,floor,roadNm,umdNm
0,108,부곡대우이안,2003,42800,1,2024,59.8516,15,부곡복지관길,삼동
1,1114,의왕내손e편한세상,2012,89300,1,2024,84.761,6,내손중앙로,내손동
2,101,한진해모로,2004,50800,1,2024,84.93,18,모락로,오전동
3,104,삼성래미안,2001,53500,1,2024,55.4831,5,포일로,내손동
4,201,인덕원 센트럴 자이 2단지,2009,71000,1,2024,59.84,11,내손로,내손동


In [14]:
# 데이터 정보 확인
df_apt_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   aptDong     1299 non-null   object
 1   aptNm       1299 non-null   object
 2   buildYear   1299 non-null   object
 3   dealAmount  1299 non-null   int64 
 4   dealMonth   1299 non-null   object
 5   dealYear    1299 non-null   object
 6   excluUseAr  1299 non-null   object
 7   floor       1299 non-null   object
 8   roadNm      1299 non-null   object
 9   umdNm       1299 non-null   object
dtypes: int64(1), object(9)
memory usage: 101.6+ KB
